In [2]:
from scipy.stats import norm
import pandas as pd
import numpy as np
from src.boot import boot_sample, compute_treatment_effects, aggregate_treatment_effects

In [3]:
def algorithm_1_with_se(data, B=1000, alpha=0.05, treatment_status: str='treatment', estimand: str='overall', groupby_column: str=None, time_column: str=None):
    """
    Implement Algorithm 1 to compute standard errors and confidence intervals for ATT.
    Also returns the standard errors.
    
    Parameters:
    - data (DataFrame): The dataset containing 'treatment', covariates, and outcomes.
    - B (int): Number of bootstrap samples.
    - alpha (float): Significance level for confidence intervals.
    - treatment_status (str): The column name for the treatment variable.
    - estimand (str): The estimand of interest. Either 'overall', 'cohort', or event.
    - groupby_column (str): The column name for the grouping variable.
    - time_column (str): The column name for the time variable.
    
    Returns:
    - CI (DataFrame): Bootstrapped simultaneous confidence band for ATT.
    - SE (Series): Standard errors for each time period.
    """
    n = len(data)

    # Compute initial ATT and treatment effects to get the number of time periods/parameters
    treatment_effects_og = compute_treatment_effects(data, estimand, groupby_column, time_column)
    aggregate_og = treatment_effects_og[treatment_effects_og[treatment_status] == 1]
    ATT = aggregate_treatment_effects(aggregate_og, 't_effects', estimand, groupby_column, time_column)

    # Convert to NumPy array and get the length
    ATT = np.array(ATT).flatten()
    t_periods = len(ATT)

    # Initialize placeholder for storing bootstrap results
    R_bootstrap = np.zeros((B, t_periods))
    t_test_bootstrap = np.zeros((B, t_periods))

    # Bootstrap sampling and calculation of R(g, t)
    for b in range(B):
        # Step 1: Draw a bootstrap sample
        bootstrap_sample = boot_sample(data)

        # Step pre-2: Compute treatment effects for bootstrap sample
        treatment_effects = compute_treatment_effects(bootstrap_sample, estimand, groupby_column, time_column)

        # Filter only treated units
        aggregate = treatment_effects[treatment_effects[treatment_status] == 1]
        
        # Step 2: Compute ATT_star for bootstrap sample
        ATT_star = aggregate_treatment_effects(aggregate, 't_effects', estimand, groupby_column, time_column)
        ATT_star = np.array(ATT_star).flatten()

        # Compute R(g, t)
        R_gt = np.sqrt(n) * (ATT - ATT_star)
        
        # Store the R(g, t) values
        R_bootstrap[b, :] = R_gt

    # Step 4: Compute bootstrap estimator of the main diagonal of Sigma^(1/2)
    q_75 = np.percentile(R_bootstrap, 75, axis=0)
    q_25 = np.percentile(R_bootstrap, 25, axis=0)
    z_75, z_25 = norm.ppf([0.75, 0.25])
    Sigma_half = (q_75 - q_25) / (z_75 - z_25)

    # Calculate standard errors
    SE = Sigma_half / np.sqrt(n)

    # Step 5: For each bootstrap draw, compute t-test
    for b in range(B):
        t_test_bootstrap[b, :] = np.abs(R_bootstrap[b, :]) / Sigma_half

    # Step 6: Construct confidence band
    c_alpha = np.percentile(t_test_bootstrap, 100 * (1 - alpha), axis=0)
    ATT_original = data.groupby(treatment_status).mean().diff().iloc[-1].filter(like='outcome')
    CI_lower = ATT_original - c_alpha * Sigma_half / np.sqrt(n)
    CI_upper = ATT_original + c_alpha * Sigma_half / np.sqrt(n)
    CI = pd.DataFrame({'Lower_Bound': CI_lower, 'Upper_Bound': CI_upper})
    
    return CI, SE

In [4]:
# Number of units, time periods, and cohorts
n_units = 10
n_time = 10
n_cohorts = 3

# Random seed for reproducibility
np.random.seed(42)

# Generate a DataFrame
df = pd.DataFrame({
    'unit_id': np.repeat(range(1, n_units + 1), n_time),
    'time_id': list(range(1, n_time + 1)) * n_units,
    'treatment': np.random.choice([0, 1], n_units * n_time),
    'covariate1': np.random.normal(0, 1, n_units * n_time),
    'covariate2': np.random.normal(0, 1, n_units * n_time),
    'cohort': np.random.choice([2010, 2011, 2012], n_units * n_time)
})

# Adding a few never-treated units
df.loc[df['unit_id'].isin([1, 2]), 'treatment'] = 0

# Simulating potential outcomes under the control (Y0)
df['Y0'] = 5 + 0.5 * df['covariate1'] + 0.3 * df['covariate2'] + 0.2 * df['unit_id'] + 0.1 * df['time_id'] + np.random.normal(0, 1, n_units * n_time)

# Defining treatment effect (constant for all units as 2)
df['treatment_effect'] = 2

# Simulating potential outcomes under the treatment (Y1)
df['Y1'] = df['Y0'] + df['treatment_effect']

# Constructing the observed outcome based on treatment status
df['outcome'] = np.where(df['treatment'] == 1, df['Y1'], df['Y0'])

# Adding pre-treatment and post-treatment periods (assuming treatment starts at time 5 for everyone)
df['period_type'] = np.where(df['time_id'] < 5, 'pre-treatment', 'post-treatment')

In [5]:
result_df = compute_treatment_effects(df, 'outcome', 'treatment', 'unit_id', 'time_id', covariates=['covariate1', 'covariate2'])

Skipping column period_type as it appears to be truly textual.


In [6]:
data = result_df.copy()
data = data[data['treatment'] == 1]
data.head()

,unit_id,time_id,treatment,covariate1,covariate2,cohort,Y0,treatment_effect,Y1,outcome,period_type,y_0,t_effects
20,3,1,1,-0.479174,-0.974682,2010,6.312380,2,8.312380,8.312380,pre-treatment,6.312380,2.0
22,3,3,1,-1.106335,1.158596,2010,5.067553,2,7.067553,7.067553,pre-treatment,5.067553,2.0
23,3,4,1,-1.196207,-0.820682,2010,3.813685,2,5.813685,5.813685,pre-treatment,3.813685,2.0
24,3,5,1,0.812526,0.963376,2012,6.469303,2,8.469303,8.469303,post-treatment,6.469303,2.0
25,3,6,1,1.356240,0.412781,2012,10.255939,2,12.255939,12.255939,post-treatment,10.255939,2.0


In [7]:
# Test the aggregate effects function
print(aggregate_treatment_effects(data, 't_effects', estimand='overall'))
print(aggregate_treatment_effects(data, 't_effects', estimand='cohort', groupby_column='cohort', time_column='time_id'))
print(aggregate_treatment_effects(data, 't_effects', estimand='event', groupby_column='cohort' ,time_column='time_id'))

1.9999999999946187
2010    2.0
2011    2.0
2012    2.0
dtype: float64
elapsed_time
-2011    2.0
-2009    2.0
-2008    2.0
-2007    2.0
-2006    2.0
-2005    2.0
-2004    2.0
-2003    2.0
-2002    2.0
-2001    2.0
-2000    2.0
Name: t_effects, dtype: float64
